# Arabic to English OCR and translation of Arabic to English of images using GoogleVisionAPI and RapidAPI in google colab and storing it in txt file.

In [ ]:
import cv2
import imutils
import json
import numpy as np
import os
import pandas as pd
import requests
from base64 import b64encode
from IPython.display import Image
from pylab import rcParams

In [ ]:
# defining figure size
rcParams['figure.figsize'] = 10, 20

In [ ]:
# to encode image into base64
def makeImageData(imgpath):
    img_req = None
    with open(imgpath, 'rb') as f:
        ctxt = b64encode(f.read()).decode()
        img_req = {
            'image': {
                'content': ctxt
            },
            'features': [{
                'type': 'DOCUMENT_TEXT_DETECTION',
                'maxResults': 1
            }],
            "imageContext": {"languageHints": ["ar-t-i0"]}
        }
    return json.dumps({"requests": img_req}).encode()

In [ ]:
# to request OCR of encoded image data
def requestOCR(url, api_key, imgpath):
  imgdata = makeImageData(imgpath)
  response = requests.post(ENDPOINT_URL, 
                           data = imgdata, 
                           params = {'key': api_key}, 
                           headers = {'Content-Type': 'application/json'}
                           )

  return response

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
ENDPOINT_URL = 'https://vision.googleapis.com/v1/images:annotate'
api_key='paste_googlevisionapi_key'
img_loc = 'arabic.jpg'

In [1]:
#Image(img_loc)

In [ ]:
# requesting OCR
result = requestOCR(ENDPOINT_URL, api_key, img_loc)

In [ ]:
if result.status_code != 200 or result.json().get('error'):
    print ("Error")
else:
    result = result.json()['responses'][0]['textAnnotations']
    print("Successfull")

In [ ]:
result

In [ ]:
arabic_ocr_txt=result[0]["description"]
print(arabic_ocr_txt)

In [ ]:
# to create the text file and write OCR into it
with open("arabic_translated.txt",'w') as writefile:
  writefile.write((result[0]["description"]))

In [ ]:
# Verify if the appending is successfully executed
with open('arabic_translated.txt', 'r') as testwritefile:
    print(testwritefile.read())

In [ ]:
# Rapid APi for translation
import requests

url = "https://translated-mymemory---translation-memory.p.rapidapi.com/api/get"
# here 'langpair' takes two parameters i.e. present language and language to be translated into.
querystring = {"q":arabic_ocr_txt,"langpair":"ar|en"}

headers = {
     'x-rapidapi-key': "paste_rapidapi_key",
     'x-rapidapi-host': "paste_host_name"
     }
response = requests.request("GET", url, headers=headers,params=querystring)

print(response.text)

In [ ]:
import re
x=re.findall('translatedText.*?:[a-zA-Z0-9]',response.text)
#print(x)
translated_t=x[0][0:-10].split(";")
print(translated_t)

In [ ]:
for i in range(len(translated_t)):
    print(translated_t[i])

In [ ]:
# to edit the created file and append the arabic-to-english translated text
with open("arabic_translated.txt",'a') as writefile:
  for i in translated_t:
    writefile.write(i+'\n')

In [ ]:
# to verify if the appending is successfully executed
with open('arabic_translated.txt', 'r') as testwritefile:
    print(testwritefile.read())